In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """
    Shuffle the deck and select five unique cards.
    """
    shuffled_deck = np.random.permutation(deck)
    hand_indices = np.random.choice(len(shuffled_deck), 5, replace=False)
    hand = shuffled_deck[hand_indices]
    sorted_hand = np.sort(hand, order='rank')
    return shuffled_deck, sorted_hand

from collections import Counter

def check_five_of_a_kind(hand):
    """Check if a hand has a five-of-a-kind and determine its case."""
    ranks = hand['rank']
    suits = hand['suit']

    # Check if all cards are the same rank
    rank_counts = Counter(ranks)
    if len(rank_counts) == 1:
        # All cards have the same rank; proceed to check suit distribution
        suit_counts = Counter(suits)

        # Case 1: One suit appears twice, and each of the other three suits appears once
        if 2 in suit_counts.values() and list(suit_counts.values()).count(1) == 3:
            return "Case 1"

        # Case 2: Two pairs of the same suit and one card with a different suit
        elif list(suit_counts.values()).count(2) == 2 and list(suit_counts.values()).count(1) == 1:
            return "Case 2"

    # If neither case is met, return None
    return None


def run_simulation(num_trials, worker_id=None):
    """
    Run simulation with enhanced random seed handling.
    Counts Five of a Kind cases with batching.
    """
    if worker_id is not None:
        # Create unique seed using large prime numbers and worker id
        base_seed = 104729  # Large prime
        multiplier = 15485863  # Another large prime
        worker_seed = base_seed + (worker_id * multiplier)
        np.random.seed(worker_seed)

    counts = Counter()
    local_examples = defaultdict(list)  # Store examples for each case

    # Process hands in smaller batches for better progress tracking
    batch_size = 100_000
    num_batches = num_trials // batch_size + (1 if num_trials % batch_size else 0)

    for batch in range(num_batches):
        current_batch_size = min(batch_size, num_trials - batch * batch_size)
        batch_counts = Counter()

        for _ in range(current_batch_size):
            _, hand = deal_hand(deck)
            case = check_five_of_a_kind(hand)
            if case:
                batch_counts[case] += 1
                # Collect up to 10 examples for each case
                if len(local_examples[case]) < 10:
                    local_examples[case].append(hand.copy())

        # Update total counts
        counts.update(batch_counts)

        if worker_id is not None:
            print(f"Worker {worker_id}: Completed batch {batch + 1}/{num_batches}, "
                  f"found Case 1: {counts['Case 1']}, "
                  f"Case 2: {counts['Case 2']} so far")

    return counts, local_examples

def run_simulation_parallel(trials, num_workers=16):
    """
    Run parallel simulation with improved worker management and progress tracking.
    """
    chunk_size = trials // num_workers
    remaining = trials % num_workers
    total_counts = Counter()
    all_examples = defaultdict(list)

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Distribute work with adjusted chunks
        futures = []
        for i in range(num_workers):
            # Last worker gets remaining hands if any
            worker_trials = chunk_size + (remaining if i == num_workers-1 else 0)
            futures.append(executor.submit(run_simulation, worker_trials, worker_id=i))

        # Track progress and collect results
        completed = 0
        results = []
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            counts, examples = future.result()
            results.append((counts, examples))

    # Combine all results
    for counts, examples in results:
        total_counts.update(counts)
        # Merge examples up to 10 per case
        for case, hands in examples.items():
            if len(all_examples[case]) < 10:
                all_examples[case].extend(hands[:10 - len(all_examples[case])])

    # Print final summary
    print("\nTotal examples collected:")
    for case in ["Case 1", "Case 2"]:
        print(f"{case}: {len(all_examples[case])} examples")

    return total_counts, all_examples

def calculate_total_five_of_a_kind_probability(counts, total_hands):
    """Calculate the total probability of getting any five-of-a-kind."""
    total_five_of_a_kind = sum(counts[case] for case in ["Case 1", "Case 2"])
    return total_five_of_a_kind / total_hands

def main():
    # Part 1: Run 100 trials, shuffle each time, and print the dealt hand
    print("\nRunning 100 trials:")
    example_hands_100 = {"Case 1": [], "Case 2": []}
    for trial in range(100):
        shuffled_deck, hand = deal_hand(deck)
        print_hand(hand)

        case = check_five_of_a_kind(hand)
        print(f"Five-of-a-kind Case: {case}")

        # Collect examples during the initial 100 trials if needed
        if case in example_hands_100 and len(example_hands_100[case]) < 10:
            example_hands_100[case].append(hand)

    # Part 2: Run full simulation for larger trial counts and calculate probability
    trial_counts = [1_00, 1_000, 10_000, 100_000, 1_000_000, 10_000_000, 100_000_000]
    for trials in trial_counts:
        print(f"\nRunning parallel simulation with {trials} trials...")
        counts, example_hands = run_simulation_parallel(trials)

        # Calculate total probability
        total_probability = calculate_total_five_of_a_kind_probability(counts, trials)

        print(f"Results for {trials} trials:")
        print(f"Case 1 (two same suit): {counts['Case 1']}")
        print(f"Case 2 (two pairs same suit): {counts['Case 2']}")
        print(f"Total probability: {total_probability:.8f}")

        # Print examples for 1,000,000 trial run
        if trials == 1_000_000:
            print("\nExample hands from 1,000,000 trials:")
            for case in ["Case 1", "Case 2"]:
                print(f"\n{case} examples:")
                for hand in example_hands[case]:
                    print_hand(hand)

if __name__ == "__main__":
    main()



Running 100 trials:
Hand: 4 of Diamonds, 4 of Clubs, 8 of Hearts, A of Hearts, A of Hearts
Five-of-a-kind Case: None
Hand: 2 of Clubs, 4 of Hearts, 6 of Diamonds, 8 of Spades, 9 of Hearts
Five-of-a-kind Case: None
Hand: 2 of Diamonds, 7 of Diamonds, 9 of Clubs, 9 of Clubs, Q of Hearts
Five-of-a-kind Case: None
Hand: 3 of Spades, 5 of Diamonds, 5 of Clubs, J of Hearts, Q of Spades
Five-of-a-kind Case: None
Hand: 2 of Hearts, 2 of Diamonds, 9 of Hearts, J of Spades, K of Hearts
Five-of-a-kind Case: None
Hand: 5 of Clubs, 6 of Spades, 8 of Clubs, 8 of Spades, A of Diamonds
Five-of-a-kind Case: None
Hand: 2 of Clubs, 5 of Diamonds, 7 of Clubs, 9 of Diamonds, Q of Clubs
Five-of-a-kind Case: None
Hand: 7 of Hearts, 7 of Spades, 9 of Hearts, Q of Spades, A of Clubs
Five-of-a-kind Case: None
Hand: 2 of Clubs, 2 of Spades, 5 of Spades, K of Clubs, K of Spades
Five-of-a-kind Case: None
Hand: 2 of Diamonds, 7 of Clubs, 8 of Hearts, 10 of Spades, A of Clubs
Five-of-a-kind Case: None
Hand: 5 of Di